# Introduction

Traditional job sites can be overwhelming. Additionally, Lambda School graduates can save time and anxiety by focusing on companies who understand the unique value Lambda graduates bring as employees. Therefore we are developing a website where Lambda School students and alumni can post company and interview experiences and find helpful posts that others have made. 

For its first user feature, the Data Science team is developing automated content moderation. As the website scales, manual content moderation may not be a pragmatic way of enforcing content rules. Furthermore, inappropriate content undermines the site's core mission of saving Lambda students time and helping ease their anxieties. Therefore we seek to find a model that can automatically and accurately classify posts that should be flagged or removed.

It is worth noting that many tweets in the data used within this notebook contain hateful or obscene content. Those who may be traumatized by such content may want to avoid reading this notebook.

## Modeling Plan

At the risk of tautology, data science requires data. So we first assess the data we have available and what we can get. We face a classic problem right now in building out features for a fledgling site: because we don't have users, we don't have actual user data; however, we'll never generate enough users to have sufficient data if they're encountering abusive and hateful content. For this reason, we concluded starting with a model trained on external data was a possible route.

We were able to find several labeled data sets that flagged hateful, abusive, or spam content (or some combination of the three). Our initial strategy will be to use these to train, validate, and test models. It is worth noting that all three data sets were composed of Tweets. Our hope is that patterns of abusive text are similar enough across websites that the model learning on this will transfer.

We will load and explore the data, then fit sequentially more complex models in order to maximize predictive performance.

# Load Data

In [2]:
import pandas as pd
import numpy as np
import spacy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import os

In [3]:
os.getcwd()

'C:\\Users\\ajenk\\GitHub\\allay-ds\\exploration'

## Data Set One: "Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior"

The first data set is a set of 100,000 tweets labeled for use in the paper "Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior." The text of these tweets has been provided for research use courtesy of the author Antigoni-Marie Founta. 

https://github.com/ENCASEH2020/hatespeech-twitter

@inproceedings{founta2018large,
    title={Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior},
    author={Founta, Antigoni-Maria and Djouvas, Constantinos and Chatzakou, Despoina and Leontiadis, Ilias and Blackburn, Jeremy and Stringhini, Gianluca and Vakali, Athena and Sirivianos, Michael and Kourtellis, Nicolas},
    booktitle={11th International Conference on Web and Social Media, ICWSM 2018},
    year={2018},
    organization={AAAI Press}
}

In [4]:
hundred_k_tweets = pd.read_csv("data\\hatespeech_text_label_vote.csv", sep='\t', header=None, names=["tweet", "category", "votes"])

In [5]:
hundred_k_tweets.head()

,tweet,category,votes
0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,spam,4
1,RT @Papapishu: Man it would fucking rule if we...,abusive,4
2,It is time to draw close to Him &#128591;&#127...,normal,4
3,if you notice me start to act different or dis...,normal,5
4,"Forget unfollowers, I believe in growing. 7 ne...",normal,3


In [6]:
hundred_k_tweets.shape

(99996, 3)

In [7]:
hundred_k_tweets['category'].value_counts()

normal     53851
abusive    27150
spam       14030
hateful     4965
Name: category, dtype: int64

For our purposes, content that is abusive, spam, or hateful should be identified by the model. The "votes" are the number of actual people who labeled the content by its majority label. This is a feature we may want to incorporate in the model down the line (examples where all five humans agreed than an item is appropriate should plausibly be weighted differently in training the model than examples where they were split). For the purposes of building a baseline model, I'll simplify data sets down to a binary target and the body of text. 

A note: emojis are characterised by their the string patterns "&#NNNNNN", where each N is a number. As advanced models are capable of learning these patterns, I leave them as is for the time being. But that's the human interpretation where those patterns are seen.

In [8]:
hundred_k_tweets["inappropriate"] = (hundred_k_tweets["category"].isin(["spam", "abusive", "hateful"]))

In [9]:
hundred_k_tweets = hundred_k_tweets.drop(["category", "votes"], axis=1)
hundred_k_tweets.head()

,tweet,inappropriate
0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,True
1,RT @Papapishu: Man it would fucking rule if we...,True
2,It is time to draw close to Him &#128591;&#127...,False
3,if you notice me start to act different or dis...,False
4,"Forget unfollowers, I believe in growing. 7 ne...",False


In [10]:
hundred_k_tweets["inappropriate"].value_counts(normalize=True)

False    0.538532
True     0.461468
Name: inappropriate, dtype: float64

## Data Set Two: Automated Hate Speech Detection and the Problem of Offensive Language

In [12]:
additional_tweets = pd.read_csv("data\\labeled_data.csv")

In [13]:
additional_tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [14]:
additional_tweets = additional_tweets.drop(["Unnamed: 0"], axis=1)

In [15]:
additional_tweets.shape

(24783, 6)

This data set is composed of 24,783 tweets that have been manually labeled by CrowdFlower users. "Count" is the number of users who voted; "hate_speech", "offensive_language", and "neither" are the various categories that can be voted for. "Class" is the majority label.

https://github.com/t-davidson/hate-speech-and-offensive-language/tree/master/data

In [16]:
additional_tweets["class"].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

"1" represents offensive language, "2" represents neither offensive language nor hate speech, and "0" represents hate speech. 

It's worth noting that inappropriate tweets are far more common in both data sets than they are in the real world. This is something to be cognizant of when training the model: weakly explanatory models will tend to default to baseline predictions, which in this case will result in many more flagged posts than is desirable. 

In [17]:
additional_tweets.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [18]:
additional_tweets.sort_values(by="offensive_language", ascending=False).head(15).T

,18766,20246,7706,23560,1600,20292,17020,4107,13926,7909,15488,4867,22464,20474,16174
count,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
hate_speech,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
offensive_language,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
neither,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
class,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
tweet,RT @chvmpagne: Bitches that dress like this pu...,RT @tupactopus: bad bitches get in free night ...,"Amennn!!!!! ""@WomenLoveBrickz: If your girlfri...","all these beautiful bitches, sucha beautiful t...","&#8220;@WEEEDITH: All I want is bitches, big b...",RT @vinnycrack: this bitch got the itunes term...,RT @RakwonOGOD: Lmaoo bitch what? http://t.co/...,@Mijo_LGI you're such a bitch,Pull up on my ex make that bitch mad,"Bad bitch, chest out......no wonder why Miss R...",RT @HilariousSelfie: how ugly bitches take sel...,"@Tee_Bizzle i aint shit, you aint shit...bitch...",Walk in the party boot up yo hoe,Ray Rice is a bitch &amp; his wife is stupid f...,RT @LeezyTheWarrior: Can't be letting them mes...


In [19]:
additional_tweets.sort_values(by="hate_speech", ascending=False).head(15).T

,6171,15658,10451,15809,3869,19136,6378,3404,23633,12949,14973,17488,5749,9357,9206
count,9,9,9,9,7,6,9,6,9,9,6,6,9,6,6
hate_speech,7,7,7,6,6,6,6,6,5,5,5,5,5,5,5
offensive_language,1,2,2,0,1,0,3,0,4,4,1,1,0,1,1
neither,1,0,0,3,0,0,0,0,0,0,0,0,4,0,0
class,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
tweet,@infidelpamelaLC I'm going to blame the black ...,RT @Isa__Lopez: @D_Lo520 but you're still a fa...,I hate fat bitches,RT @JihadistJoe: We Muslims have no military h...,@L1LTR4P fucking losers wetbacks #SorryNotSorry,RT @iBeZo: Stupid fucking nigger LeBron. You f...,@kcSnowWhite7 @SamSaunders42 don't forget napp...,@Hovaa_ ya I know all the slang I'm racist I h...,"bitch kill yoself, go on to the bathroom and e...",My advice of the day: If your a tranny.....go ...,RT @DefendWallSt: Tell me how you really feel....,RT @SwaaggyyV: Fucking chinks in Clash of Clan...,"@clinchmtn316 @sixonesixband AMERICA today, th...",GEEZ..... I think #NorthKorea may be right. #B...,"From now on, I will call all radical MUSLIMS n..."


In [20]:
additional_tweets.sort_values(by="hate_speech", ascending=True).head(15).T

,0,15719,15718,15717,15715,15713,15712,15710,15709,15708,15705,15704,15703,15702,15700
count,3,4,3,3,3,3,3,3,3,3,3,3,3,3,3
hate_speech,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
offensive_language,0,1,1,2,1,0,3,3,3,3,3,0,3,3,3
neither,3,3,2,1,2,3,0,0,0,0,0,3,0,0,0
class,2,2,2,1,2,2,1,1,1,1,1,2,1,1,1
tweet,!!! RT @mayasolovely: As a woman you shouldn't...,RT @JZolly23: Kickin trash cans on the golf ca...,"RT @JStac825: There's coon classic (R. Kelly, ...","RT @JRsBBQ: TV wrestling villains must lie, se...",RT @JOscarJr: @paullemat Happy Birthday! Take ...,RT @JOEL9ONE: Thanks Carolina fans 4 the flipp...,"RT @JMuggaaa: ""@TheKaosYatti: &#8220;@JMuggaaa...",RT @JMK0728: Lmao...........poor pussy!!!!! ht...,RT @JLewyville: &#128563; @Treslyon: Keyair an...,RT @JLM_2014: It ain't nothin to cut that bitc...,RT @JFlocka: Need a down bitch to bring me pizza,RT @JFish13: Where has this been all year? But...,RT @JETzLyfe412: Started off wit nuttin I was ...,RT @JEN_JEN_2014: My pussy is totes adorbs whe...,RT @JDYDFF: know the bitch before you call you...


In [21]:
additional_tweets["inappropriate"] = additional_tweets["class"] != 2

In [22]:
additional_tweets.head()

,count,hate_speech,offensive_language,neither,class,tweet,inappropriate
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,False
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,True
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,True
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,True
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,True


In [23]:
additional_tweets = additional_tweets.drop(["count", "hate_speech", "offensive_language", "neither", "class"], axis=1)

In [24]:
additional_tweets.head()

,tweet,inappropriate
0,!!! RT @mayasolovely: As a woman you shouldn't...,False
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,True
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,True
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,True
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,True


In [25]:
additional_tweets["inappropriate"].value_counts()

True     20620
False     4163
Name: inappropriate, dtype: int64

## Data Set 3: Kaggle Tweets - https://www.kaggle.com/vkrahul/twitter-hate-speech#train_E6oV3lV.csv

In [26]:
kaggle_tweets = pd.read_csv("data/train_E6oV3lV.csv")

In [27]:
kaggle_tweets.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [28]:
kaggle_tweets["inappropriate"] = kaggle_tweets["label"]
kaggle_tweets = kaggle_tweets.drop(["id", "label"], axis=1)

In [29]:
kaggle_tweets["inappropriate"].value_counts()

0    29720
1     2242
Name: inappropriate, dtype: int64

In [30]:
kaggle_tweets.head()

,tweet,inappropriate
0,@user when a father is dysfunctional and is s...,0
1,@user @user thanks for #lyft credit i can't us...,0
2,bihday your majesty,0
3,#model i love u take with u all the time in ...,0
4,factsguide: society now #motivation,0


Lastly, we add in a data set of Tweets sourced from Kaggle. This data set has many more appropriate than inappropriate items, which is nice to counterbalance the last data set.

# Merging Data

In [31]:
dfs = [hundred_k_tweets, additional_tweets, kaggle_tweets]
df = pd.concat(dfs, ignore_index=True)

In [32]:
df.shape

(156741, 2)

Some of the data has duplication. Some duplicate tweets have identical labels -- in these cases, we simply drop one. Other duplicate tweets have contradictory labels. These are all inappropriate tweets, so we drop the ones labeled appropriate on these.

In [33]:
df = df.drop_duplicates(subset=['tweet', 'inappropriate'])
appropriate = ~df['inappropriate']
dupe_tweet = df.duplicated(subset=['tweet'], keep=False)
df = df[~((dupe_tweet) & (appropriate))].copy()

In [36]:
df.head()

,tweet,inappropriate
0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,1
1,RT @Papapishu: Man it would fucking rule if we...,1
2,It is time to draw close to Him &#128591;&#127...,0
3,if you notice me start to act different or dis...,0
4,"Forget unfollowers, I believe in growing. 7 ne...",0


In [37]:
df.duplicated(subset=['tweet']).any()

False

In [35]:
# df.to_csv('combined_deduped.csv', index=False)

## Model Training

Now that we have the data in a csv file, we can move on to model training. The notebook exploring the model training process is in exploration/train_models.ipynb.